In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
import json
import re
from openai import OpenAI
from langchain.llms.base import LLM
from groq import Groq
from typing import List, Optional
import time
from tqdm import tqdm

In [2]:
# Short description:
short_similarity = pd.read_excel("D:\\CodingSystem\\notebooks\\bupa_mapped_short_services_bert.xlsx")

# Long Description:
long_similarity = pd.read_excel("D:\\CodingSystem\\notebooks\\bupa_mapped_long_services_bert.xlsx")

short_similarity.shape, long_similarity.shape

((2484, 3), (2484, 3))

In [3]:
class FireworksLLM(LLM):
    model: str
    api_key: str
    base_url: str = "https://api.fireworks.ai/inference/v1"
    temperature: float = 0
    top_p: float = 0

    @property
    def _llm_type(self) -> str:
        return "fireworks"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        client = OpenAI(
            api_key=self.api_key,
            base_url=self.base_url
        )

        response = client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are an expert in medical coding and service mapping."},
                {"role": "user", "content": prompt}
            ],
            temperature=self.temperature,
            top_p=self.top_p
        )
        return response.choices[0].message.content

load_dotenv()

api_key = os.getenv("FIREWORKS_API_KEY")

fireworks_llm = FireworksLLM(
    model="accounts/fireworks/models/qwen3-235b-a22b",
    api_key=api_key,
)

In [4]:
def safe_json_parse(response_text: str):
    """
    Extract and parse ONLY the first valid JSON object or array.
    """
    cleaned = response_text.strip()

    # Remove accidental ```json or ``` 
    cleaned = re.sub(r"^```json|```$", "", cleaned, flags=re.IGNORECASE).strip()

    # Extract first JSON array
    match = re.search(r"(\[.*?\])", cleaned, re.DOTALL)
    if not match:
        raise ValueError("No valid JSON array found in LLM response.")

    json_str = match.group(1)
    return json.loads(json_str)

def safe_json_parse_row_validation(response_text: str):
    """
    Strip code fences, trim, and parse JSON safely.
    """
    cleaned = response_text.strip()
    # Remove code fences if any
    if cleaned.startswith("```"):
        cleaned = cleaned.strip("```").strip()
    # Try to find first { ... }
    if not cleaned.startswith("{"):
        first_brace = cleaned.find("{")
        if first_brace != -1:
            cleaned = cleaned[first_brace:]
    return json.loads(cleaned)

def sanitize_description(text):
    """
    Clean up extra whitespace and brackets if needed.
    """
    if pd.isna(text):
        return ""
    return str(text).strip().replace("\n", " ")

# Short Validaion:

In [ ]:
def validate_mappings_batched_resume(
    full_df: pd.DataFrame,
    llm,
    checkpoint_file: str = "bupa_llm_validation_results.csv",
    batch_size: int = 10
):
    """
    Validate pairs in batches, resume from checkpoint,
    and save failed batches' rows for later inspection.
    """

    # Load checkpoint if exists
    try:
        results_df = pd.read_csv(checkpoint_file)
        validated_indices = set(results_df["row_idx"])
        print(f"✅ Found checkpoint: {len(validated_indices)} rows already validated.")
    except FileNotFoundError:
        results_df = pd.DataFrame(columns=[
            "row_idx", "AHJ_DESCRIPTION", "SBS_SHORT_DESCRIPTION",
            "Similarity", "LLM_Answer", "Reason"
        ])
        validated_indices = set()
        print(f"🆕 No checkpoint found. Starting fresh!")

    unvalidated_mask = ~full_df.index.isin(validated_indices)
    to_validate_df = full_df[unvalidated_mask]

    print(f"🔍 Total rows: {len(full_df)}")
    print(f"✅ Already validated: {len(validated_indices)}")
    print(f"⚡ Still to validate: {len(to_validate_df)}")

    all_results = results_df.to_dict(orient="records")
    failed_rows = []

    def process_batch(start, end, batch):
        nonlocal validated_indices, all_results, failed_rows

        prompt = (
            "You are an expert in medical coding and service mapping.\n\n"
            "Below is a list of AHJ_DESCRIPTION and SBS_SHORT_DESCRIPTION pairs.\n\n"
            "For each pair:\n"
            "- Decide if the AHJ_DESCRIPTION correctly maps to the SBS_SHORT_DESCRIPTION,\n"
            "  OR if the SBS_SHORT_DESCRIPTION is a valid generalization, broader category, or plural of the AHJ_DESCRIPTION,\n"
            "  OR if the AHJ_DESCRIPTION is a valid generalization, broader category, or plural of the SBS_SHORT_DESCRIPTION.\n\n"
            "Important:\n"
            "• Be careful: 'unilateral' and 'bilateral' are different and should NOT be treated the same.\n"
            "• Small punctuation differences such as (hyphens, brackets, commas) do not affect the meaning.\n"
            "• If you are unsure, answer NO. Do not guess.\n\n"
            "Examples:\n"
            "✔ 'LEFT KNEE ARTHROSCOPY' vs. 'KNEE ARTHROSCOPY' → YES (valid generalization)\n"
            "✔ 'PORCELAIN FUSED TO METAL CROWN' vs. 'PORCELAIN FUSED TO BASE METAL CROWN' → YES (valid generalization)\n"
            "✔ 'ENDOSCOPIC BANDING OF OESOPHAGEAL VARICE' vs. 'ENDOSCOPIC BANDING OF OESOPHAGEAL VARICES' → YES (plural)\n"
            "❌ 'UNILATERAL MASTECTOMY' vs. 'BILATERAL MASTECTOMY' → NO (different laterality)\n\n"
            "✅ You MUST follow these rules:\n"
            "1. Return only valid JSON.\n"
            "2. Do NOT include ```json, markdown, or any other text.\n"
            "3. Return ONLY the JSON array and nothing else.\n"
            "4. ✅ Return ONLY a single valid JSON object and nothing else.\n"
            "5. ❌ Do not return plain text or an array.\n"
            "6. Format: [{\"index\": 0, \"answer\": \"YES\", \"reason\": \"...\"}, ...]\n\n"
            "If you break these rules, you will fail.\n\n"
            "Pairs:\n"
        )

        for i, (_, row) in enumerate(batch.iterrows()):
            ahj_desc = sanitize_description(row['AHJ_DESCRIPTION'])
            sbs_desc = sanitize_description(row['SBS_SHORT_DESCRIPTION'])
            prompt += f"{i})\n"
            prompt += f"- AHJ_DESCRIPTION: \"{ahj_desc}\"\n"
            prompt += f"- SBS_SHORT_DESCRIPTION: \"{sbs_desc}\"\n\n"

        answers = None
        max_retries = 3

        for attempt in range(max_retries):
            try:
                response_text = llm._call(prompt)
                print(f"🔍 Raw LLM response for batch {start}-{end}:\n{response_text}\n")  # Debug!
                answers = safe_json_parse(response_text)
                break
            except Exception as e:
                print(f"⚠️ Error for batch {start}-{end} attempt {attempt+1}: {e}")
                if attempt < max_retries - 1:
                    print(f"🔁 Retrying batch {start}-{end} (attempt {attempt+2}/{max_retries})...")
                    time.sleep(5)

        if answers is None:
            print(f"❌ Batch {start}-{end} failed after {max_retries} attempts. Saving rows to failed_rows.")
            failed_rows.extend(batch.index.values.tolist())
            return

        for i, ans in enumerate(answers):
            idx = batch.index[i]
            row = batch.iloc[i]
            all_results.append({
                "row_idx": idx,
                "AHJ_DESCRIPTION": row["AHJ_DESCRIPTION"],
                "SBS_SHORT_DESCRIPTION": row["SBS_SHORT_DESCRIPTION"],
                "Similarity": row.get("SHORT_SIMILARITY_SCORE", ""),
                "LLM_Answer": ans["answer"].strip().upper(),
                "Reason": ans.get("reason", "").strip()
            })
            validated_indices.add(idx)

        pd.DataFrame(all_results).to_csv(checkpoint_file, index=False)
        print(f"✅ Saved checkpoint: {len(validated_indices)} rows validated.")
        time.sleep(1)

    for start in tqdm(range(0, len(to_validate_df), batch_size)):
        end = min(start + batch_size, len(to_validate_df))
        batch = to_validate_df.iloc[start:end].copy()
        process_batch(start, end, batch)

    if failed_rows:
        failed_df = full_df.loc[failed_rows]
        failed_df.to_csv("bupa_failed_rows.csv", index_label="row_idx")
        print(f"💾 Saved {len(failed_rows)} failed rows to bupa_failed_rows.csv for later retry or review.")

    final_df = pd.DataFrame(all_results).sort_values("row_idx")
    return final_df

In [10]:
short_results_df = validate_mappings_batched_resume(
    short_similarity,   # your input dataframe
    fireworks_llm,           # your LLM instance
    checkpoint_file="bupa_llm_validation_results.csv",
    batch_size=10       # Smaller batch is safer!
)

✅ Found checkpoint: 1460 rows already validated.
🔍 Total rows: 2484
✅ Already validated: 1460
⚡ Still to validate: 1024


  0%|          | 0/103 [00:00<?, ?it/s]

⚠️ Error for batch 0-10 attempt 1: No valid JSON found in response
🔁 Retrying batch 0-10 (attempt 2/3)...
⚠️ Error for batch 0-10 attempt 2: No valid JSON found in response
🔁 Retrying batch 0-10 (attempt 3/3)...


  1%|          | 1/103 [01:36<2:44:27, 96.74s/it]

⚠️ Error for batch 0-10 attempt 3: No valid JSON found in response
❌ Batch 0-10 failed after 3 attempts. Saving rows to failed_rows.
⚠️ Error for batch 10-20 attempt 1: No valid JSON found in response
🔁 Retrying batch 10-20 (attempt 2/3)...
⚠️ Error for batch 10-20 attempt 2: No valid JSON found in response
🔁 Retrying batch 10-20 (attempt 3/3)...


...

✅ Saved checkpoint: 2410 rows validated.


 99%|█████████▉| 102/103 [47:06<00:14, 14.86s/it]

✅ Saved checkpoint: 2414 rows validated.


100%|██████████| 103/103 [47:16<00:00, 27.54s/it]

💾 Saved 70 failed rows to bupa_failed_rows.csv for later retry or review.


In [25]:
def retry_failed_rows(
    llm,
    failed_file: str = "bupa_failed_rows.csv",
    checkpoint_file: str = "bupa_llm_validation_results.csv"
):
    """
    Retry failed rows one by one, sanitize text, and update the checkpoint.
    """
    failed_df = pd.read_csv(failed_file, index_col="row_idx")
    results_df = pd.read_csv(checkpoint_file)
    validated_indices = set(results_df["row_idx"])
    all_results = results_df.to_dict(orient="records")

    print(f"🔁 Retrying {len(failed_df)} failed rows...")

    for idx, row in failed_df.iterrows():
        if idx in validated_indices:
            continue

        ahj_desc = sanitize_description(row["AHJ_DESCRIPTION"])
        sbs_desc = sanitize_description(row["SBS_SHORT_DESCRIPTION"])

        # single_prompt = (
        #     "You are an expert in medical coding and service mapping.\n\n"
        #     "Below is one pair:\n"
        #     "- Decide if the AHJ_DESCRIPTION correctly maps to the SBS_SHORT_DESCRIPTION,\n"
        #     "  or if the SBS_SHORT_DESCRIPTION is a valid generalization or plural of the AHJ_DESCRIPTION (falls under its category) and vice versa.\n"
        #     "- Be careful \"unilateral\" and \"bilateral\" are different.\n"
        #     "- Answer YES or NO and explain briefly.\n"
        #     "- Return ONLY the word 'YES' or 'NO' at the start of your answer.\n\n"
        #     "Return your answer in JSON format like this:\n"
        #     "{\"answer\": \"YES\", \"reason\": \"Fully matches.\"}\n\n"
        #     "✅ Return ONLY valid JSON and nothing else. Do not add any explanation before or after the JSON.\n\n"
        #     f"- AHJ_DESCRIPTION: \"{ahj_desc}\"\n"
        #     f"- SBS_SHORT_DESCRIPTION: \"{sbs_desc}\"\n"
        # )

        single_prompt = (
            "You are an expert in medical coding and service mapping.\n\n"
            "Below is a list of AHJ_DESCRIPTION and SBS_SHORT_DESCRIPTION pairs.\n\n"
            "For each pair:\n"
            "- Decide if the AHJ_DESCRIPTION correctly maps to the SBS_SHORT_DESCRIPTION,\n"
            "  OR if the SBS_SHORT_DESCRIPTION is a valid generalization, broader category, or plural of the AHJ_DESCRIPTION,\n"
            "  OR if the AHJ_DESCRIPTION is a valid generalization, broader category, or plural of the SBS_SHORT_DESCRIPTION.\n\n"
            "Important:\n"
            "• Be careful: 'unilateral' and 'bilateral' are different and should NOT be treated the same.\n"
            "• Small punctuation differences such as (hyphens, brackets, commas) do not affect the meaning.\n"
            "• If you are unsure, answer NO. Do not guess.\n\n"
            "Examples:\n"
            "✔ 'LEFT KNEE ARTHROSCOPY' vs. 'KNEE ARTHROSCOPY' → YES (valid generalization)\n"
            "✔ 'PORCELAIN FUSED TO METAL CROWN' vs. 'PORCELAIN FUSED TO BASE METAL CROWN' → YES (valid generalization)\n"
            "✔ 'ENDOSCOPIC BANDING OF OESOPHAGEAL VARICE' vs. 'ENDOSCOPIC BANDING OF OESOPHAGEAL VARICES' → YES (plural)\n"
            "❌ 'UNILATERAL MASTECTOMY' vs. 'BILATERAL MASTECTOMY' → NO (different laterality)\n\n"
            "✅ You MUST follow these rules:\n"
            "1. Return only valid JSON.\n"
            "2. Do NOT include ```json, markdown, or any other text.\n"
            "3. Return ONLY the JSON array and nothing else.\n"
            "4. ✅ Return ONLY a single valid JSON object and nothing else.\n"
            "5. ❌ Do not return plain text or an array.\n"
            "6. Format: [{\"index\": 0, \"answer\": \"YES\", \"reason\": \"...\"}, ...]\n\n"
            "If you break these rules, you will fail.\n\n"
            f"- AHJ_DESCRIPTION: \"{ahj_desc}\"\n"
            f"- SBS_SHORT_DESCRIPTION: \"{sbs_desc}\"\n"
        )

        answers = None
        max_retries = 3
        for attempt in range(max_retries):
            try:
                response_text = llm._call(single_prompt)
                answers = safe_json_parse(response_text)
                break
            except Exception as e:
                print(f"⚠️ Error for row {idx} attempt {attempt+1}: {e}")
                time.sleep(3)

        if answers is None:
            print(f"❌ Row {idx} failed again after {max_retries} attempts.")
            continue

        # Defensive: handle accidental array
        if isinstance(answers, list):
            answers = answers[0]

        all_results.append({
            "row_idx": idx,
            "AHJ_DESCRIPTION": row["AHJ_DESCRIPTION"],
            "SBS_SHORT_DESCRIPTION": row["SBS_SHORT_DESCRIPTION"],
            "Similarity": row.get("SHORT_SIMILARITY_SCORE", ""),
            "LLM_Answer": answers.get("answer", "").strip().upper(),
            "Reason": answers.get("reason", "").strip()
        })
        validated_indices.add(idx)

        pd.DataFrame(all_results).to_csv(checkpoint_file, index=False)
        print(f"✅ Saved single-row checkpoint: {len(validated_indices)} rows validated.")

    print(f"🎉 Retry complete! Check {checkpoint_file} for updated results.")

In [26]:
# Example call for Fireworks or Groq LLM
resent_short_results_df = retry_failed_rows(
    llm=fireworks_llm,  # or groq_llm or any other LLM instance you built
    failed_file="bupa_failed_rows.csv",
    checkpoint_file="bupa_llm_validation_results.csv"
)

🔁 Retrying 70 failed rows...
✅ Saved single-row checkpoint: 2483 rows validated.
✅ Saved single-row checkpoint: 2484 rows validated.
🎉 Retry complete! Check bupa_llm_validation_results.csv for updated results.


In [28]:
short_validation = pd.read_csv("bupa_short_llm_validation_results.csv")
print(short_validation.shape)
short_validation.head()

(2484, 6)


,row_idx,AHJ_DESCRIPTION,SBS_SHORT_DESCRIPTION,Similarity,LLM_Answer,Reason
0,0,Duplex u/s extracranial/carotid & vert,DUPLEX U/S EXTRACRANIAL/CAROTID & VERT,1.000000,YES,Fully matches except for case and spacing.
1,1,total arthroplasty of knee unilateral,TOTAL ARTHROPLASTY OF KNEE UNILATERAL,1.000000,YES,Fully matches except for spacing.
2,2,ENDOSCOPIC BANDING OF OESOPHAGEAL VARICE,ENDOSCOPIC BANDING OF OESOPHAGEAL VARICES,0.997117,YES,Pluralization of 'VARICE' to 'VARICES' is valid.
3,3,"DUPLEX U/S ART/BYPS GRAFTS UPP LMB, UNI",DUPLEX U/S ART/BYPS GRAFTS UPP LMB UNI,0.996361,YES,Matches except for punctuation spacing.
4,4,"CONDUCTN STUD, EMG SGL FIBRES NRV & MUSC",CONDUCTN STUD EMG SGL FIBRES NRV & MUSC,0.996189,YES,Comma vs space difference does not affect mean...


In [34]:
short_wrong_mapped_services = list(short_validation[short_validation['LLM_Answer'] == 'NO']['AHJ_DESCRIPTION'].unique())
len(short_wrong_mapped_services)

2024

In [32]:
yes_short_index = short_validation[short_validation['LLM_Answer'] == 'YES']['row_idx'].values
no_short_index = short_validation[short_validation['LLM_Answer'] == 'NO']['row_idx'].values

len(yes_short_index), len(no_short_index)

(460, 2024)

In [33]:
bupa_valid_short_mappings = short_similarity.loc[yes_short_index]
bupa_valid_short_mappings.shape

(460, 3)

In [35]:
bupa_valid_short_mappings.to_excel('bupa_valid_short_mappings.xlsx')

# Long Validation:

In [5]:
def validate_long_mappings_batched_resume(
    full_df: pd.DataFrame,
    llm,
    checkpoint_file: str = "bupa_long_llm_validation_results.csv",
    batch_size: int = 10
):
    """
    Validate pairs in batches, resume from checkpoint,
    and save failed batches' rows for later inspection.
    """

    # Load checkpoint if exists
    try:
        results_df = pd.read_csv(checkpoint_file)
        validated_indices = set(results_df["row_idx"])
        print(f"✅ Found checkpoint: {len(validated_indices)} rows already validated.")
    except FileNotFoundError:
        results_df = pd.DataFrame(columns=[
            "row_idx", "AHJ_DESCRIPTION", "SBS_LONG_DESCRIPTION",
            "Similarity", "LLM_Answer", "Reason"
        ])
        validated_indices = set()
        print(f"🆕 No checkpoint found. Starting fresh!")

    unvalidated_mask = ~full_df.index.isin(validated_indices)
    to_validate_df = full_df[unvalidated_mask]

    print(f"🔍 Total rows: {len(full_df)}")
    print(f"✅ Already validated: {len(validated_indices)}")
    print(f"⚡ Still to validate: {len(to_validate_df)}")

    all_results = results_df.to_dict(orient="records")
    failed_rows = []

    def process_batch(start, end, batch):
        nonlocal validated_indices, all_results, failed_rows

        prompt = (
            "You are an expert in medical coding and service mapping.\n\n"
            "Below is a list of AHJ_DESCRIPTION and SBS_LONG_DESCRIPTION pairs.\n\n"
            "For each pair:\n"
            "- Decide if the AHJ_DESCRIPTION correctly maps to the SBS_LONG_DESCRIPTION,\n"
            "  OR if the SBS_LONG_DESCRIPTION is a valid generalization, broader category, or plural of the AHJ_DESCRIPTION,\n"
            "  OR if the AHJ_DESCRIPTION is a valid generalization, broader category, or plural of the SBS_LONG_DESCRIPTION.\n\n"
            "Important:\n"
            "• Be careful: 'unilateral' and 'bilateral' are different and should NOT be treated the same.\n"
            "• Singularity and Plurality don't affect the meaning.\n"
            "• Small punctuation differences (hyphens, brackets, commas) do not affect the meaning.\n"
            "• If you are unsure, answer NO. Do not guess.\n\n"
            "Examples:\n"
            "✔ 'LEFT KNEE ARTHROSCOPY' vs. 'KNEE ARTHROSCOPY' → YES (valid generalization)\n"
            "✔ 'PORCELAIN FUSED TO METAL CROWN' vs. 'PORCELAIN FUSED TO BASE METAL CROWN' → YES (valid generalization)\n"
            "✔ 'BILATERAL MASTECTOMY' vs. 'MASTECTOMY' → YES (valid generalization)\n"
            "✔ 'TOOTH EXTRACTION' vs. 'TEETH EXTRACTIONS' → YES (plural)\n"
            "❌ 'UNILATERAL MASTECTOMY' vs. 'BILATERAL MASTECTOMY' → NO (different laterality)\n\n"
            "✅ You MUST follow these rules:\n"
            "Answer YES or NO and explain briefly for each pair.\n\n"
            "Return your answers in strict JSON array format like this:\n"
            "[{\"index\": 0, \"answer\": \"YES\", \"reason\": \"Fully matches.\"}, ...]\n\n"
            "✅ Return ONLY a valid JSON array and nothing else. \n\n"
            "❌ Do NOT add any text, comment, or markdown before or after it. \n\n"
            "Pairs:\n"
        )

        for i, (_, row) in enumerate(batch.iterrows()):
            ahj_desc = sanitize_description(row['AHJ_DESCRIPTION'])
            sbs_desc = sanitize_description(row['SBS_LONG_DESCRIPTION'])
            prompt += f"{i})\n"
            prompt += f"- AHJ_DESCRIPTION: \"{ahj_desc}\"\n"
            prompt += f"- SBS_LONG_DESCRIPTION: \"{sbs_desc}\"\n\n"

        answers = None
        max_retries = 3

        for attempt in range(max_retries):
            try:
                response_text = llm._call(prompt)
                answers = safe_json_parse(response_text)
                break
            except Exception as e:
                print(f"⚠️ Error for batch {start}-{end} attempt {attempt+1}: {e}")
                if attempt < max_retries - 1:
                    print(f"🔁 Retrying batch {start}-{end} (attempt {attempt+2}/{max_retries})...")
                    time.sleep(5)

        if answers is None:
            print(f"❌ Batch {start}-{end} failed after {max_retries} attempts. Saving rows to failed_rows.")
            failed_rows.extend(batch.index.values.tolist())
            return

        for i, ans in enumerate(answers):
            idx = batch.index[i]
            row = batch.iloc[i]
            all_results.append({
                "row_idx": idx,
                "AHJ_DESCRIPTION": row["AHJ_DESCRIPTION"],
                "SBS_LONG_DESCRIPTION": row["SBS_LONG_DESCRIPTION"],
                "Similarity": row.get("LONG_SIMILARITY_SCORE", ""),
                "LLM_Answer": ans["answer"].strip().upper(),
                "Reason": ans.get("reason", "").strip()
            })
            validated_indices.add(idx)

        pd.DataFrame(all_results).to_csv(checkpoint_file, index=False)
        print(f"✅ Saved checkpoint: {len(validated_indices)} rows validated.")
        time.sleep(1)

    # Main loop: no retries here
    for start in tqdm(range(0, len(to_validate_df), batch_size)):
        end = min(start + batch_size, len(to_validate_df))
        batch = to_validate_df.iloc[start:end].copy()
        process_batch(start, end, batch)

    # Save failed rows to separate file
    if failed_rows:
        failed_df = full_df.loc[failed_rows]
        failed_df.to_csv("bupa_failed_rows.csv", index_label="row_idx")
        print(f"💾 Saved {len(failed_rows)} failed rows to bupa_failed_rows.csv for later retry or review.")

    final_df = pd.DataFrame(all_results).sort_values("row_idx")
    return final_df

In [6]:
long_results_df = validate_long_mappings_batched_resume(
    long_similarity,   # your input dataframe
    fireworks_llm,           # your LLM instance
    checkpoint_file="bupa_long_llm_validation_results.csv",
    batch_size=10       # Smaller batch is safer!
)

✅ Found checkpoint: 1040 rows already validated.
🔍 Total rows: 2484
✅ Already validated: 1040
⚡ Still to validate: 1444


  0%|          | 0/145 [00:00<?, ?it/s]

⚠️ Error for batch 0-10 attempt 1: No valid JSON array found in LLM response.
🔁 Retrying batch 0-10 (attempt 2/3)...
✅ Saved checkpoint: 1050 rows validated.


  1%|          | 1/145 [01:11<2:51:48, 71.58s/it]

⚠️ Error for batch 10-20 attempt 1: No valid JSON array found in LLM response.
🔁 Retrying batch 10-20 (attempt 2/3)...
⚠️ Error for batch 10-20 attempt 2: No valid JSON array found in LLM response.
🔁 Retrying batch 10-20 (attempt 3/3)...


...

✅ Saved checkpoint: 2040 rows validated.


 99%|█████████▉| 144/145 [1:43:15<00:26, 26.29s/it]

⚠️ Error for batch 1440-1444 attempt 1: Expecting value: line 1 column 2 (char 1)
🔁 Retrying batch 1440-1444 (attempt 2/3)...
⚠️ Error for batch 1440-1444 attempt 2: Expecting value: line 1 column 2 (char 1)
🔁 Retrying batch 1440-1444 (attempt 3/3)...


100%|██████████| 145/145 [1:43:48<00:00, 42.96s/it]

⚠️ Error for batch 1440-1444 attempt 3: Expecting value: line 1 column 2 (char 1)
❌ Batch 1440-1444 failed after 3 attempts. Saving rows to failed_rows.
💾 Saved 444 failed rows to bupa_failed_rows.csv for later retry or review.


In [7]:
def retry_failed_rows(
    llm,
    failed_file: str = "bupa_failed_rows.csv",
    checkpoint_file: str = "bupa_long_llm_validation_results.csv"
):
    """
    Retry failed rows one by one, sanitize text, and update the checkpoint.
    """
    failed_df = pd.read_csv(failed_file, index_col="row_idx")
    results_df = pd.read_csv(checkpoint_file)
    validated_indices = set(results_df["row_idx"])
    all_results = results_df.to_dict(orient="records")

    print(f"🔁 Retrying {len(failed_df)} failed rows...")

    for idx, row in failed_df.iterrows():
        if idx in validated_indices:
            continue

        ahj_desc = sanitize_description(row["AHJ_DESCRIPTION"])
        sbs_desc = sanitize_description(row["SBS_LONG_DESCRIPTION"])

        single_prompt = (
            "You are an expert in medical coding and service mapping.\n\n"
            "Below is a list of AHJ_DESCRIPTION and SBS_LONG_DESCRIPTION pairs.\n\n"
            "For each pair:\n"
            "- Decide if the AHJ_DESCRIPTION correctly maps to the SBS_LONG_DESCRIPTION,\n"
            "  OR if the SBS_LONG_DESCRIPTION is a valid generalization, broader category, or plural of the AHJ_DESCRIPTION,\n"
            "  OR if the AHJ_DESCRIPTION is a valid generalization, broader category, or plural of the SBS_LONG_DESCRIPTION.\n\n"
            "Important:\n"
            "• Be careful: 'unilateral' and 'bilateral' are different and should NOT be treated the same.\n"
            "• Small punctuation differences such as (hyphens, brackets, commas) do not affect the meaning.\n"
            "• If you are unsure, answer NO. Do not guess.\n\n"
            "Examples:\n"
            "✔ 'LEFT KNEE ARTHROSCOPY' vs. 'KNEE ARTHROSCOPY' → YES (valid generalization)\n"
            "✔ 'PORCELAIN FUSED TO METAL CROWN' vs. 'PORCELAIN FUSED TO BASE METAL CROWN' → YES (valid generalization)\n"
            "✔ 'BILATERAL MASTECTOMY' vs. 'MASTECTOMY' → YES (valid generalization)\n"
            "✔ 'TOOTH EXTRACTION' vs. 'TEETH EXTRACTIONS' → YES (plural)\n"
            "❌ 'UNILATERAL MASTECTOMY' vs. 'BILATERAL MASTECTOMY' → NO (different laterality)\n\n"
            "✅ You MUST follow these rules:\n"
            "1. Return only valid JSON.\n"
            "2. Do NOT include ```json, markdown, or any other text.\n"
            "3. Return ONLY the JSON array and nothing else.\n"
            "4. ✅ Return ONLY a single valid JSON object and nothing else.\n"
            "5. ❌ Do not return plain text or an array.\n"
            "6. Format: [{\"index\": 0, \"answer\": \"YES\", \"reason\": \"...\"}, ...]\n\n"
            "If you break these rules, you will fail.\n\n"
            f"- AHJ_DESCRIPTION: \"{ahj_desc}\"\n"
            f"- SBS_LONG_DESCRIPTION: \"{sbs_desc}\"\n"
        )

        answers = None
        max_retries = 3
        for attempt in range(max_retries):
            try:
                response_text = llm._call(single_prompt)
                answers = safe_json_parse_row_validation(response_text)
                break
            except Exception as e:
                print(f"⚠️ Error for row {idx} attempt {attempt+1}: {e}")
                time.sleep(3)

        if answers is None:
            print(f"❌ Row {idx} failed again after {max_retries} attempts.")
            continue

        # Defensive: handle accidental array
        if isinstance(answers, list):
            answers = answers[0]

        all_results.append({
            "row_idx": idx,
            "AHJ_DESCRIPTION": row["AHJ_DESCRIPTION"],
            "SBS_LONG_DESCRIPTION": row["SBS_LONG_DESCRIPTION"],
            "Similarity": row.get("SHORT_LONG_SCORE", ""),
            "LLM_Answer": answers.get("answer", "").strip().upper(),
            "Reason": answers.get("reason", "").strip()
        })
        validated_indices.add(idx)

        pd.DataFrame(all_results).to_csv(checkpoint_file, index=False)
        print(f"✅ Saved single-row checkpoint: {len(validated_indices)} rows validated.")

    print(f"🎉 Retry complete! Check {checkpoint_file} for updated results.")

In [8]:
# Example call for Fireworks or Groq LLM
resent_long_results_df = retry_failed_rows(
    llm=fireworks_llm,
    failed_file="bupa_failed_rows.csv",
    checkpoint_file="bupa_long_llm_validation_results.csv"
)

🔁 Retrying 444 failed rows...
⚠️ Error for row 190 attempt 1: Expecting value: line 1 column 1 (char 0)
✅ Saved single-row checkpoint: 2041 rows validated.
⚠️ Error for row 191 attempt 1: Expecting value: line 1 column 1 (char 0)
✅ Saved single-row checkpoint: 2042 rows validated.
✅ Saved single-row checkpoint: 2043 rows validated.
✅ Saved single-row checkpoint: 2044 rows validated.
✅ Saved single-row checkpoint: 2045 rows validated.
✅ Saved single-row checkpoint: 2046 rows validated.
✅ Saved single-row checkpoint: 2047 rows validated.
⚠️ Error for row 197 attempt 1: Expecting value: line 1 column 1 (char 0)
✅ Saved single-row checkpoint: 2048 rows validated.
✅ Saved single-row checkpoint: 2049 rows validated.
✅ Saved single-row checkpoint: 2050 rows validated.
✅ Saved single-row checkpoint: 2051 rows validated.
⚠️ Error for row 211 attempt 1: Expecting value: line 1 column 1 (char 0)
✅ Saved single-row checkpoint: 2052 rows validated.
✅ Saved single-row checkpoint: 2053 rows validated

# Check RAG Mappings:

In [3]:
full_mappings = pd.read_excel("D:/CodingSystem/notebooks/full_mappings.xlsx")
full_mappings.shape

(92236, 14)

In [4]:
full_mappings.head()

,INSURANCE_COMPANY,SERVICE_CODE,SERVICE_DESCRIPTION,PRICE,SERVICE_KEY,SERVICE_CLASSIFICATION,SERVICE_CATEGORY,SBS Code,SBS Code (Hyphenated),SHORT_DESCRIPTION,Long Description,Definition,Chapter Name,Block Name
0,Al Rajhi,LA0009004,ORAL GLUCOSE TOLERANCE TEST,438.600,76961,LAB Services,LAB-Biochemistry,665420020,66542-00-20,ORAL GLUCOSE TOLERANCE TEST,ORAL GLUCOSE TOLERANCE TEST,A test that measures the body's response to su...,"Non-invasive, cognitive and other intervention...",Physiological assessment
1,Al Rajhi,XY0077772,RADIOGRAPHY OF ABDOMEN,246.708,81526,RAD Services,X-Ray,589000090,58900-00-90,RADIOGRAPHY ABDO,RADIOGRAPHY OF ABDOMEN,NaN,Imaging services,Radiography of abdomen
2,Al Rajhi,XY0077746,MAGNETIC RESONANCE ANGIOGRAPHY OF SPINE,1477.476,81537,RAD Services,MRA,909020300,90902-03-00,MAGNETIC RESONANCE ANGIOGRAPHY OF SPINE,MAGNETIC RESONANCE ANGIOGRAPHY OF SPINE,NaN,Imaging services,Magnetic resonance angiography
3,Al Rajhi,XY0077753,"RADIOGRAPHY OF ANKLE AND FOOT, UNILATERAL",314.160,81544,RAD Services,X-Ray,575240401,57524-04-01,RADIOGRAPHY ANKLE & FOOT UNI,"RADIOGRAPHY OF ANKLE AND FOOT, UNILATERAL",Radiography of ankle and foot (one side).,Imaging services,Radiography of lower limb
4,Al Rajhi,XY0078124,"RADIOGRAPHY OF ELBOW AND HUMERUS, UNILATERAL",267.960,81554,RAD Services,X-Ray,575120001,57512-00-01,RADIOGRAPHY ELBOW & HUMERUS UNI,"RADIOGRAPHY OF ELBOW AND HUMERUS, UNILATERAL","Radiography of elbow and humerus (one side), e...",Imaging services,Radiography of upper limb


In [5]:
mapped_services = pd.read_excel("D:/CodingSystem/assets/mapped_services.xlsx")
mapped_services.shape

(8544, 5)

In [6]:
mapped_services.head()

,SERVICE_CODE,SERVICE_DESCRIPTION,SBS_CODE,SBS_DESCRIPTION,LLM_EXPLAINATION
0,LA0013674,AFP**,73050-01-70,AFP (TOTAL); SERUM,The internal service (LAB-Hormones) aligns wit...
1,LA0043519,RESPIRATORY INFECTIONS PANEL (VIRAL-BACTERIAL)...,73050-57-70,RESPIRATORY VIRUS (3-5); AMPLIFIED PROBE,The internal service uses PCR (a nucleic acid ...
2,LA0043329,TISSUE TRANSGLUTAMINASE IGG ABS,73100-16-75,TISSUE TRANSGLUTAMINASE AB,This code explicitly includes assays for both ...
3,LA0043328,TISSUE TRANSGLUTAMINASE IGA ABS,73050-11-50,HUMAN TISSUE TRANSGLUTAMINASE,This code explicitly specifies measurement of ...
4,LA0027629,ENTERO VIRUS (COXSACKIE) ABS,73100-21-40,ENTEROVIRUS AB,The SBS code 73100-21-40 directly matches the ...


In [7]:
llm_mapped_services = full_mappings[full_mappings['SERVICE_DESCRIPTION'].isin(list(mapped_services['SERVICE_DESCRIPTION'].unique()))]
llm_mapped_services.drop(columns=['INSURANCE_COMPANY', 'PRICE', 'SERVICE_KEY'], inplace= True)
llm_mapped_services.drop_duplicates(inplace=True)
llm_mapped_services.shape

C:\Users\doha.ramadan\AppData\Local\Temp\ipykernel_23172\1031212365.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  llm_mapped_services.drop(columns=['INSURANCE_COMPANY', 'PRICE', 'SERVICE_KEY'], inplace= True)
C:\Users\doha.ramadan\AppData\Local\Temp\ipykernel_23172\1031212365.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  llm_mapped_services.drop_duplicates(inplace=True)


(8471, 11)

In [8]:
llm_mapped_services.head()

,SERVICE_CODE,SERVICE_DESCRIPTION,SERVICE_CLASSIFICATION,SERVICE_CATEGORY,SBS Code,SBS Code (Hyphenated),SHORT_DESCRIPTION,Long Description,Definition,Chapter Name,Block Name
1741,LA0013674,AFP**,LAB Services,LAB-Hormones,730500170,73050-01-70,AFP (TOTAL); SERUM,MEASUREMENT OF TOTAL ALPHA FETOPROTEIN (AFP) I...,This test is for the quantitation of alpha–fet...,Laboratory and Pathology,Chemistry & Microbiology
1742,LA0027636,HEAVY METALS IN WATER,LAB Services,LAB-Biochemistry,730501790,73050-17-90,HEAVY METAL; QUANT.,QUANTITATIVE MEASUREMENT OF HEAVY METAL,Measures the amount of heavy metals (e.g. arse...,Laboratory and Pathology,Chemistry & Microbiology
1744,LA0027634,FRAGILE X SYNDROME(FMR1) BY PCR,LAB Services,Lab-Hematology,733500162,73350-01-62,FMR1 GENE; EVALUATION,FRAGILE X MENTAL RETARDATION 1 (FMR1) EXPANDED...,Testing for abnormal expanded alleles (e.g. fr...,Laboratory and Pathology,Molecular Pathology including Gene Sequencing
1745,LA0027633,FOOD BACTERIOLOGICAL ASSESSMENT,Microbiology,Microbiology,730504090,73050-40-90,CULTURE SCREEN,SCREENING FOR CULTURE,Screening for specific pathogenic microorganis...,Laboratory and Pathology,Chemistry & Microbiology
1746,LA0027632,FIBROMAX,LAB Services,LAB-Biochemistry,730502120,73050-21-20,IRON BINDING CAPACITY,MEASUREMENT OF IRON BINDING CAPACITY IN SERUM,Measurement of iron binding capacity (transfer...,Laboratory and Pathology,Chemistry & Microbiology


In [18]:
class FireworksLLM(LLM):
    model: str
    api_key: str
    base_url: str = "https://api.fireworks.ai/inference/v1"
    temperature: float = 0
    top_p: float = 0

    @property
    def _llm_type(self) -> str:
        return "fireworks"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        client = OpenAI(
            api_key=self.api_key,
            base_url=self.base_url
        )

        response = client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are an expert in medical coding and service mapping."},
                {"role": "user", "content": prompt}
            ],
            temperature=self.temperature,
            top_p=self.top_p
        )
        return response.choices[0].message.content

load_dotenv()

api_key = os.getenv("FIREWORKS_API_KEY")

fireworks_llm = FireworksLLM(
    model="accounts/fireworks/models/deepseek-v3-0324",
    api_key=api_key,
)

In [13]:
prompt_template = """
You are an expert medical coding validator. 
Check if the AHJ service is correctly mapped to the SBS code.

AHJ SERVICE:
- Code: {SERVICE_CODE}
- Description: {SERVICE_DESCRIPTION}
- Classification: {SERVICE_CLASSIFICATION}
- Category: {SERVICE_CATEGORY}

SBS CODE:
- Code: {SBS_Code}
- Hyphenated Code: {SBS_Code_Hyphenated}
- Short Description: {SHORT_DESCRIPTION}
- Long Description: {Long_Description}
- Definition: {Definition}
- Chapter Name: {Chapter_Name}
- Block Name: {Block_Name}

Validation Rules:
- The descriptions must match semantically.
- The classification and category must align with the SBS Chapter and Block.
- There should be no obvious mismatches (e.g., test matrix conflict).
- Be strict: if unsure, return 0.

Output ONLY 1 if correctly mapped, 0 if it’s mismapped.

Your final answer must be ONLY a single number: 
1 if mapped correctly, 0 if not.

Do NOT add any explanation or chain-of-thought.
Output format: just 1 or 0.
"""

In [22]:
# Checkpoint file name
checkpoint_file = 'validated_checkpoint.csv'

# If checkpoint exists, load it; otherwise create a new column
if os.path.exists(checkpoint_file):
    validated_df = pd.read_csv(checkpoint_file)
    print(f"✅ Loaded checkpoint with {validated_df['VALIDATION_RESULT'].notnull().sum()} validated rows.")
else:
    llm_mapped_services['VALIDATION_RESULT'] = None
    validated_df = llm_mapped_services.copy()

# ✅ Loop with tqdm and safe parsing
for idx, row in tqdm(validated_df.iterrows(), total=len(validated_df), desc="Validating Services"):
    if pd.notnull(row['VALIDATION_RESULT']):
        continue  # Already validated

    prompt = prompt_template.format(
        SERVICE_CODE=row['SERVICE_CODE'],
        SERVICE_DESCRIPTION=row['SERVICE_DESCRIPTION'],
        SERVICE_CLASSIFICATION=row['SERVICE_CLASSIFICATION'],
        SERVICE_CATEGORY=row['SERVICE_CATEGORY'],
        SBS_Code=row['SBS Code'],
        SBS_Code_Hyphenated=row['SBS Code (Hyphenated)'],
        SHORT_DESCRIPTION=row['SHORT_DESCRIPTION'],
        Long_Description=row['Long Description'],
        Definition=row['Definition'],
        Chapter_Name=row['Chapter Name'],
        Block_Name=row['Block Name']
    )

    try:
        response = fireworks_llm._call(prompt)

        # ⚡ NEW robust parsing:
        lines = [line.strip() for line in response.strip().splitlines() if line.strip()]
        last_line = lines[-1] if lines else ""
        response_clean = ''.join(filter(str.isdigit, last_line))
        if response_clean in ['0', '1']:
            result_int = int(response_clean)
        else:
            print(f"⚠️ Unexpected output at index {idx}: {response.strip()}")
            result_int = None

    except Exception as e:
        print(f"⚠️ Error at index {idx}: {e}")
        result_int = None

    validated_df.at[idx, 'VALIDATION_RESULT'] = result_int
    validated_df.to_csv(checkpoint_file, index=False)

print("✅ Validation complete! Saving final file...")

✅ Loaded checkpoint with 2078 validated rows.


Validating Services:  28%|██▊       | 2366/8471 [14:42<37:58,  2.68it/s]    


PermissionError: [Errno 13] Permission denied: 'validated_checkpoint.csv'